<a href="https://colab.research.google.com/github/jassynavarro/CCADMACL_EXERCISES/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [325]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [326]:
housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)

1. Use any filter method to select the best features

In [327]:
df_housing_features = pd.DataFrame(housing.data, columns=housing.feature_names)
df_housing_target = pd.DataFrame(housing.target, columns=['target'])

df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [328]:
# put your answer here
from sklearn.feature_selection import f_classif

threshold = 4 # the number of most relevant features
high_score_features = []
feature_scores = f_classif(housing.data, housing.target.values.ravel())[0]


for score, f_name in sorted(zip(feature_scores, housing.data.columns), reverse=True)[:threshold]:
      # print(f_name, score)
      high_score_features.append(f_name)

df_housing_fc = housing.data[high_score_features]
df_housing_fc.columns

Index(['MedInc', 'Latitude', 'Population', 'HouseAge'], dtype='object')

In [329]:
import numpy as np
from sklearn.model_selection import train_test_split

X = df_housing_fc
y = housing.target

# Split dataset to select feature and evaluate the classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [330]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
rmse_filter = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse_filter)

RMSE: 0.812321794634539


2. Use any wrapper method to select the best features

In [331]:
# put your answer here
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

threshold = 5 # the number of most relevant features
model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
selector = RFE(model_rf, n_features_to_select=5, step=1)

selector = selector.fit(df_housing_features, housing.target.values.ravel())
selector_ind = selector.get_support()
df_housing_rfe = df_housing_features.iloc[:, selector_ind]
df_housing_rfe.columns

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'Latitude'], dtype='object')

In [332]:
X = df_housing_rfe
y = housing.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [333]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
rmse_wrapper = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse_wrapper)

RMSE: 0.8101195272957092


3. Use any embedded methood to select the best features

In [334]:
X = housing.data
y = housing.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [335]:
# put your answer here
from sklearn.feature_selection import SelectFromModel

model_rf = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_rf.fit(df_housing_features, housing.target.values.ravel())

sel_sfm = SelectFromModel(model_rf, prefit=True)
sel_sfm_index = sel_sfm.get_support()
df_housing_sfm = df_housing_features.iloc[:, sel_sfm_index]
df_housing_sfm.columns

Index(['MedInc', 'AveOccup'], dtype='object')

In [336]:
y_pred = model_rf.predict(X_test)

rmse_embedded = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse_embedded)

RMSE: 0.7806839804980642


In [337]:
X = housing.data
y = housing.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [338]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 0.7273129773603114


In [339]:
print("RMSE (Default):", rmse)
print("RMSE (Filter Method):", rmse_filter)
print("RMSE (Wrapper Method):", rmse_wrapper)
print("RMSE (Embedded Method):", rmse_embedded)

RMSE (Default): 0.7273129773603114
RMSE (Filter Method): 0.812321794634539
RMSE (Wrapper Method): 0.8101195272957092
RMSE (Embedded Method): 0.7806839804980642
